<a href="https://colab.research.google.com/github/moaz9090/AI-based-threat-detection-system/blob/main/GP_NIDS_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
from fastai.tabular.all import df_shrink

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Moaz_GP/test.csv')

<ipython-input-35-6b37d152c71d>:1: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/Moaz_GP/test.csv')


In [ ]:
data

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
0,192.168.100.148,65389.0,192.168.100.7,80.0,6.0,7.0,420.0,3.0,0.0,0.0,...,0.0,512.0,0.0,35840.0,140.0,0.0,0.0,0.0,0.0,1.0
1,192.168.100.148,11154.0,192.168.100.5,80.0,6.0,7.0,280.0,2.0,40.0,1.0,...,0.0,512.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,192.168.1.31,42062.0,192.168.1.79,1041.0,6.0,0.0,44.0,1.0,40.0,1.0,...,0.0,1024.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,192.168.1.34,46849.0,192.168.1.79,9110.0,6.0,0.0,44.0,1.0,40.0,1.0,...,0.0,1024.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,192.168.1.30,50360.0,192.168.1.152,1084.0,6.0,0.0,44.0,1.0,40.0,1.0,...,0.0,1024.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
# Load the saved model
model = load_model('/content/drive/MyDrive/Moaz_GP/NIDS_model.h5')

In [ ]:
data = data.drop(columns=['IPV4_SRC_ADDR', 'IPV4_DST_ADDR'])

In [ ]:
data = df_shrink(data, obj2cat=False, int2uint=False)

In [ ]:
data.replace([np.inf, -np.inf], np.nan, inplace=True)    
print(data.isna().any(axis=1).sum(), "rows with at least one NaN to remove")
data.dropna(inplace=True)

71200 rows with at least one NaN to remove


In [ ]:
print(data.duplicated().sum(), "fully duplicate rows to remove")
data.drop_duplicates(inplace=True)
data.reset_index(inplace=True, drop=True)
data.shape
data

68 fully duplicate rows to remove


,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
0,65389.0,80.0,6.0,7.000,420.0,3.0,0.0,0.0,2.0,2.0,...,0.0,512.0,0.0,35840.0,140.0,0.0,0.0,0.0,0.0,1.0
1,11154.0,80.0,6.0,7.000,280.0,2.0,40.0,1.0,22.0,2.0,...,0.0,512.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,42062.0,1041.0,6.0,0.000,44.0,1.0,40.0,1.0,22.0,2.0,...,0.0,1024.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,46849.0,9110.0,6.0,0.000,44.0,1.0,40.0,1.0,22.0,2.0,...,0.0,1024.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,50360.0,1084.0,6.0,0.000,44.0,1.0,40.0,1.0,22.0,2.0,...,0.0,1024.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69927,33863.0,80.0,6.0,7.000,280.0,2.0,0.0,0.0,2.0,2.0,...,0.0,512.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
69928,11643.0,80.0,17.0,188.000,56.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
69929,49510.0,80.0,6.0,7.680,447.0,6.0,1815.0,5.0,27.0,27.0,...,1.0,8192.0,29200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69930,59460.0,35735.0,6.0,0.000,44.0,1.0,40.0,1.0,22.0,2.0,...,0.0,1024.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
data.drop(columns=['L4_SRC_PORT','L4_DST_PORT'], axis=1, inplace=True)

In [ ]:
print('Min values:\n',data.describe().loc['min'])

Min values:
 PROTOCOL                        1.0
L7_PROTO                        0.0
IN_BYTES                        4.0
IN_PKTS                         1.0
OUT_BYTES                       0.0
OUT_PKTS                        0.0
TCP_FLAGS                       0.0
CLIENT_TCP_FLAGS                0.0
SERVER_TCP_FLAGS                0.0
FLOW_DURATION_MILLISECONDS      0.0
DURATION_IN                     0.0
DURATION_OUT                    0.0
MIN_TTL                         0.0
MAX_TTL                         0.0
LONGEST_FLOW_PKT               28.0
SHORTEST_FLOW_PKT              28.0
MIN_IP_PKT_LEN                  0.0
MAX_IP_PKT_LEN                 28.0
SRC_TO_DST_SECOND_BYTES         0.0
DST_TO_SRC_SECOND_BYTES         0.0
RETRANSMITTED_IN_BYTES          0.0
RETRANSMITTED_IN_PKTS           0.0
RETRANSMITTED_OUT_BYTES         0.0
RETRANSMITTED_OUT_PKTS          0.0
SRC_TO_DST_AVG_THROUGHPUT       0.0
DST_TO_SRC_AVG_THROUGHPUT       0.0
NUM_PKTS_UP_TO_128_BYTES        0.0
NUM_PKTS_128_TO

In [ ]:

print('Max values:\n',data.describe().loc['max'])

Max values:
 PROTOCOL                       2.530000e+02
L7_PROTO                       2.440000e+02
IN_BYTES                       6.493320e+06
IN_PKTS                        1.082220e+05
OUT_BYTES                      3.506066e+07
OUT_PKTS                       2.382500e+04
TCP_FLAGS                      2.230000e+02
CLIENT_TCP_FLAGS               2.230000e+02
SERVER_TCP_FLAGS               2.220000e+02
FLOW_DURATION_MILLISECONDS     4.294966e+06
DURATION_IN                    1.031510e+05
DURATION_OUT                   6.507000e+03
MIN_TTL                        2.550000e+02
MAX_TTL                        2.550000e+02
LONGEST_FLOW_PKT               3.127000e+03
SHORTEST_FLOW_PKT              1.478000e+03
MIN_IP_PKT_LEN                 4.220000e+02
MAX_IP_PKT_LEN                 3.127000e+03
SRC_TO_DST_SECOND_BYTES        2.120000e+17
DST_TO_SRC_SECOND_BYTES        4.810000e+22
RETRANSMITTED_IN_BYTES         5.313040e+05
RETRANSMITTED_IN_PKTS          3.770000e+02
RETRANSMITTED_OUT_B

In [ ]:
labels = data['Label']
data = data.drop('Label' , axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

def predict_and_evaluate(model, data, labels, threshold_quantile, mae=True):
    if mae:
        # MAE loss
        losses = np.mean(np.abs(data - model.predict(data)), axis=1)
    else:
        # MSE loss
        losses = np.mean((data - model.predict(data)) ** 2, axis=1)

    # Define the threshold based on the supplied quantile
    threshold = np.quantile(losses, threshold_quantile)
    
    # Making predictions
    preds = np.array(losses > threshold, dtype=np.uint0)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

    return accuracy*100

In [ ]:
predict_and_evaluate(model, data , labels , threshold_quantile=0.35 , mae=True)

2186/2186 [==============================] - 3s 2ms/step


70.57713207115484

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def predict_and_evaluate2(model, data, true_labels, threshold_quantile, mae=True):
    if mae:
        # MAE loss
        losses = np.mean(np.abs(data - model.predict(data)), axis=1)
    else:
        # MSE loss
        losses = np.mean((data - model.predict(data)) ** 2, axis=1)

    # Define the threshold based on the supplied quantile
    threshold = np.quantile(losses, threshold_quantile)
    
    # Making predictions
    preds = np.array(losses > threshold, dtype=np.uint0)
    
    # Calculate metrics
    accuracy = accuracy_score(true_labels, preds)
    precision = precision_score(true_labels, preds)
    recall = recall_score(true_labels, preds)
    f1 = f1_score(true_labels, preds)

    return accuracy, precision, recall, f1

In [ ]:
predict_and_evaluate2(model, data,labels, threshold_quantile=0.35, mae=False)

2186/2186 [==============================] - 4s 2ms/step


(0.7173254018189098, 0.7973204857444561, 0.774456173340741, 0.7857220283131354)